In [ ]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm

In [ ]:
data = json.load(open("./handled/item2attributes.json", "r"))

In [ ]:
example_dict = {}
for item_dict in tqdm(data.values()):
    example_dict.update(item_dict)

In [ ]:
example_dict["details"]

In [ ]:
example_dict.keys()

rank: "106,074inClothing,Shoesamp;Jewelry(" 我觉得使用in把字符串分开，然后只用排名数字

date: 实际上是打分

asin: item的标识符，直接去掉

description: 物品描述，是一个列表。要控制长度。建议只使用列表中第一个

fit: 去掉

also_buy: 去掉

feature: 也是一个列表。拼接起来，但要注意长度

also_view: 去掉

similar_item: 去掉

tech1: 也是一个列表，类似于feature。也需要拼接起来，但要注意长度。

title, brand, price, details直接使用

In [ ]:
instruction = "The fashion item has the following attributes: \n "

In [ ]:
item_data = {}
for item_dict in tqdm(data.values()):
    item_prompt = copy.deepcopy(instruction)
    item_id = None
    for key, value in item_dict.items():
        if key in ["fit", "also_buy", "also_view", "similar_item", "imageURL", "imageURLHighRes"]:   # drop longitude and latitude
            continue
        elif key in ["asin"]:  # get the item id
            item_id = value
        elif key in ["description", "feature", "tech1"]:    # list type attributes
            attri_str = ""
            for meta_str in value:
                attri_str += (meta_str + ", ")
            if len(value) == 0:
                attri_str = "none, "
            attri_str = attri_str.replace("\n", " ").replace(";", ".")
            if len(attri_str) > 100:
                attri_str = attri_str[:100]
            attri_prompt = key + " is " + attri_str[:-2] + "; "    # [:-2] is to remove the last ", "
            item_prompt += attri_prompt
        else:   # str type attributes 
            if len(value) > 100:
                value = value[:100]
            attri_prompt = key + " is " + str(value).replace("\n", " ").replace(";", ".") + "; "
            item_prompt += attri_prompt
    if item_id:
        item_data[item_id] = item_prompt[:-2]
    else:
        raise ValueError("No item id")

In [ ]:
json.dump(item_data, open("./handled/item_str_0722.json", "w"))

In [ ]:
# convert to jsonline
def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open("./handled/"+ data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

id_map = json.load(open("./handled/id_map.json", "r"))["item2id"]
json_data = []
for key, value in item_data.items():
    json_data.append({"input": value, "target": "", "item": key, "item_id": id_map[key]})

save_data("item_str_0722.jsonline", json_data)